# Finding a Particular Value in Nested Dictionary

### Problem Statement: 
We are given a nested json file, with unknown structure.  It has typical key-value pairs, but some of the values are nested key-value pairs.  But there is no guidance upfront on how many keys at each level, nor is there guidance on how many levels.  But we are looking for a particular "value" (or perhaps key-value) ... whether it exists at all ... and ultimately how to find it.

### Solutions:
* Regular expressions (I don't like this approach, because it won't preserve the key-value structure)
* Convert the value to a string
* Recursive function
* flatten_json

### Example nested dictionary below
In the nested dictionary below, find a key-value pair: 
>'tomorrow': 'future'

In [1]:
# simple nested dictionary ... goal is to find value 'future'
nested_dict_sample = {
    'a': 1,
    'b': 2,
    'c': 3,
    'd': {
        'alpha': 'first',
        'beta': 'second',
        'gamma': {
            'yesterday': -1,
            'today': 0,
            'tomorrow': 'future'
        }
    }
}

In [3]:
# examine the keys at the top level
nested_dict_sample.keys()

dict_keys(['a', 'b', 'c', 'd'])

In [4]:
# examine value for one of the keys
nested_dict_sample['a']

1

In [5]:
# using dict.items() only unpacks the first level ... list of tuple pairs 
# first item in the tuple is the key, the second item is the value
# this would work for un-nested dictionaries
nested_dict_sample.items()

dict_items([('a', 1), ('b', 2), ('c', 3), ('d', {'alpha': 'first', 'beta': 'second', 'gamma': {'yesterday': -1, 'today': 0, 'tomorrow': 'future'}})])

In [15]:
# convert the key-value pairs to strings ... so you can only know which top-level key it is in
# https://stackoverflow.com/questions/14849293/python-find-index-position-in-list-based-of-partial-string
indices = [i for i, s in enumerate(nested_dict_sample.items()) if 'future' in str(s)]

# this will print the entire value ... might be huge if lots of nested levels
for each_index in indices:
    print('The string we are looking for is somewhere inside here: ')
    print(list(nested_dict_sample.items())[each_index])

The string we are looking for is somewhere inside here: 
('d', {'alpha': 'first', 'beta': 'second', 'gamma': {'yesterday': -1, 'today': 0, 'tomorrow': 'future'}})


In [16]:
# this is a simple boolean result for whether the search string is inside the dictionary at all ... not where
any('future' in str(x) for x in nested_dict_sample.items())

True

### using recursive function

https://stackoverflow.com/questions/14962485/finding-a-key-recursively-in-a-dictionary

In [24]:
def get_recursively(search_dict, field):
    """
    Takes a dict with nested lists and dicts,
    and searches all dicts for a key of the field
    provided.
    
    Made a few changes:
     * using dict.items() instead of dict.iteritems()
     * searching for value instead of key
    
    
    References: 
    https://stackoverflow.com/questions/14962485/finding-a-key-recursively-in-a-dictionary
    """
    fields_found = []

    for each_tuple in search_dict.items():
        
        (key, value) = each_tuple

        if value == field:
            fields_found.append(value)

        elif isinstance(value, dict):
            results = get_recursively(value, field)
            for result in results:
                fields_found.append(result)

        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    more_results = get_recursively(item, field)
                    for another_result in more_results:
                        fields_found.append(another_result)

    return fields_found

In [25]:
get_recursively(nested_dict_sample, 'future')

['future']

### flatten_json

Once the json is flatten, just search using list comprehension on dict.items().

In [28]:
! pip install --upgrade pip
! pip install flatten_json

    100% |████████████████████████████████| 1.4MB 10.1MB/s ta 0:00:01
  Found existing installation: pip 18.1
    Uninstalling pip-18.1:
      Successfully uninstalled pip-18.1


In [29]:
import flatten_json

In [32]:
flatten_sample = flatten_json.flatten(nested_dict_sample)
flatten_sample

{'a': 1,
 'b': 2,
 'c': 3,
 'd_alpha': 'first',
 'd_beta': 'second',
 'd_gamma_yesterday': -1,
 'd_gamma_today': 0,
 'd_gamma_tomorrow': 'future'}

In [39]:
str_search = 'future'
# str_search = 'not here'
[(k, v) for k, v in flatten_sample.items() if v == str_search]

[('d_gamma_tomorrow', 'future')]

### Other solutions:
https://stackoverflow.com/questions/31033549/nested-dictionary-value-from-key-path
    
benedict
https://github.com/fabiocaccamo/python-benedict

dpath-python
https://github.com/akesterson/dpath-python